In [547]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error
from numpy import round

In [548]:
df_pure = pd.read_csv('ecom_processed.csv', sep=',', decimal=".")
data = pd.DataFrame()
data["month"] = df_pure["month"].unique()
df = df_pure.dropna()
df.head()

,Unnamed: 0,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,payer,time_of_day,final_price
0,0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,1,ночь,9999.0
20,20,771147371911,United States,Android,organic,2019-05-06 15:49:16,2019-05-06 15:53:16,240,2019-05-06,5,1,15,2019-05-06 15:49:16,4999.0,Mobile payments,0.0,1,день,4999.0
28,28,245941309860,Germany,Mac,organic,2019-05-07 12:00:49,2019-05-07 13:10:26,4177,2019-05-07,5,2,13,2019-05-07 12:01:43,4999.0,Mobile payments,1.0,1,день,4499.0
33,33,25672200561,United States,Android,organic,2019-05-09 16:40:44,2019-05-09 16:41:31,47,2019-05-09,5,4,16,2019-05-09 16:40:44,4999.0,Debit card,1.0,1,день,4499.0
34,34,241867611399,United States,PC,социальные сети,2019-05-09 09:23:22,2019-05-09 09:28:28,306,2019-05-09,5,4,9,2019-05-09 09:23:24,4999.0,Mobile payments,0.0,1,утро,4999.0


In [549]:
def func(price):
    prices = df["final_price"].unique()
    return min(prices, key=lambda x:abs(x-price))

### Регрессионная модель 1: прогнозируем сумму покупки

In [550]:
ohe = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")
df["device"] = ohe.fit_transform(df[["device"]])
factors = df[["sessiondurationsec", "device", "day"]]
target = df["final_price"]
x_train, x_test, y_train, y_test = train_test_split(factors, target, random_state=42, test_size=0.25)
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
prediction = lin_reg.predict(x_test)
prediction = list(map(func, list(prediction)))
print(f"MAPE: {round(mean_absolute_percentage_error(y_test, prediction), 5)}")

MAPE: 0.10926


/var/folders/25/m60d5xvj3xv0532z7xrhjnl00000gp/T/ipykernel_69865/3267150004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["device"] = ohe.fit_transform(df[["device"]])


Факторами, позволяющими определить сумму покупки оказались продолжительность сессии, устройство и день недели

In [551]:
data["users"] = data["month"].apply(lambda x: len(df_pure.loc[df_pure["month"] == x]))
data["purchases"] = data["month"].apply(lambda x: len(df_pure.loc[(df_pure["month"] == x) & (df_pure["payer"] == 1)]))
data["united_states"] = data["month"].apply(lambda x: len(df_pure.loc[(df_pure["month"] == x) & (df_pure["region"] == "United States")]))
data.head(6)

,month,users,purchases,united_states
0,5,146,32,105
1,6,154,49,97
2,7,144,42,100
3,8,187,69,124
4,9,184,37,123
5,10,182,46,128


### Регрессионная модель 2: прогнозируем количество покупателей за месяц

In [552]:
factors = data[["users", "united_states"]]
target = data["purchases"]
x_train, x_test, y_train, y_test = train_test_split(factors, target, random_state=42, test_size=0.20)
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
prediction = lin_reg.predict(x_test)
prediction = round(prediction)
print(f"MAPE: {round(mean_absolute_percentage_error(y_test, prediction), 5)}")

MAPE: 0.13393
